In [1]:
import pandas as pd
import numpy as np
import jieba
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer

### 将同一个id的多行的数据，变为一行一个id的数据

In [79]:
def get_deafult():
    return defaultdict(lambda: 3)
table = defaultdict(get_deafult)

In [80]:
with open('../data/train.csv', encoding='utf-8') as f:
    f.readline()
    for line in f.readlines():
        content_id, content, subject, sentiment_value, sentiment_word = line.split(',')
        table['content'][content_id] = content
        table[subject][content_id] = int(sentiment_value)+1
        word = 'word_{}'.format(subject)
        # table[word][content_id] = sentiment_word

In [81]:
df = pd.DataFrame.from_dict(table)

In [82]:
df.head(2)

,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
df.fillna(3, inplace=True)
df.rename_axis('content_id', inplace=True)

In [84]:
df.head(2)

,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
content_id,,,,,,,,,,,
01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0
01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [85]:
df.to_csv('../data/input.csv')

In [ ]:

# df = pd.read_csv('../data/train.csv')

# df.groupby('subject').describe()

In [ ]:

# word_set = set()


# # In[7]:


# for line in df['content']:
#     words = jieba.cut(line)
#     word_set.update(words)


# # In[10]:


# in_count = 0
# with open('../data/merge_sgns_bigram_char300.txt', encoding='utf-8') as f, open('../data/cliped_vec.vec', 'w', encoding='utf-8') as outf:
#     f.readline()
#     while True:
#         line = f.readline()
#         if not line:
#             break
#         try:
#             word, vec = line.split(' ', 1)
#         except:
#             print(line)
#             continue
#         if word in word_set:
#             in_count += 1
#             outf.write(line)
#         else:
#             print(word)
# print('total have ', in_count, 'words in word embedding')


### 得到每个文档中tfidf比较重要的term，去重之后作为０/１特征

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer

In [26]:
df = pd.read_csv('../data/input.csv')
df.head(5)

,content_id,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
0,01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0
1,01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,02VHu4amZkpAGTb7,常用配件价格不贵，跟大众差不多。而且常用配件都有备货，如果不太常用的可能需要等，但也绝不是从...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,038JPMTIu1tRcZ2p,4万公里，悲催的11。。。市区每天办事，堵成汪。 哈哈哈。如果以前的锐志，起码13~14,3.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0
4,03AUEJcWODxb9ZHa,建议最好使用原厂，subaru的发动机的冷却方式更别的车不一样，所以原厂更适合。,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [27]:
def load_stopwords(filepath):
    with open(filepath, encoding='utf-8') as f:
        stops = set([v.strip() for v in f.readlines()])
    return stops
stops = load_stopwords('../data/stopwords.txt')
def seg_stop_merge(text):
    words = set(jieba.cut(text))
    words -= stops
    text = ' '.join(list(words))
    return text

In [28]:
test_df = pd.read_csv('../data/test_public.csv')
test_df.head(3)

,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜


In [29]:
train_corpus = df['content'].apply(lambda text: seg_stop_merge(text))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.868 seconds.
Prefix dict has been built succesfully.


In [30]:
test_corpus = test_df['content'].apply(lambda text: seg_stop_merge(text))

In [31]:
corpus = pd.concat([train_corpus, test_corpus])

In [32]:
vectorizer = CountVectorizer()

In [33]:
X = vectorizer.fit_transform(corpus)

In [34]:
word = vectorizer.get_feature_names()

In [35]:
# X.toarray()

In [36]:
transformer = TfidfTransformer()
transformer

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [37]:
tfidf = transformer.fit_transform(X)

In [38]:
# tfidf.toarray()

In [39]:
term_weights = []
word = vectorizer.get_feature_names()#获取词袋模型中的所有词语
weight = tfidf.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
for i in range(len(weight)):#打印每类文本的tf-idf词语权重，第一个for遍历所有文本，第二个for便利某一类文本下的词语权重
    print("-------这里输出第",i,u"类文本的词语tf-idf权重------")
    for j in range(len(word)):
        # print(word[j],weight[i][j])
        if weight[i][j] > 0:
            term_weights.append((word[j], weight[i][j]))


-------这里输出第 0 类文本的词语tf-idf权重------
-------这里输出第 1 类文本的词语tf-idf权重------
-------这里输出第 2 类文本的词语tf-idf权重------
-------这里输出第 3 类文本的词语tf-idf权重------
-------这里输出第 4 类文本的词语tf-idf权重------
-------这里输出第 5 类文本的词语tf-idf权重------
-------这里输出第 6 类文本的词语tf-idf权重------
-------这里输出第 7 类文本的词语tf-idf权重------
-------这里输出第 8 类文本的词语tf-idf权重------
-------这里输出第 9 类文本的词语tf-idf权重------
-------这里输出第 10 类文本的词语tf-idf权重------
-------这里输出第 11 类文本的词语tf-idf权重------
-------这里输出第 12 类文本的词语tf-idf权重------
-------这里输出第 13 类文本的词语tf-idf权重------
-------这里输出第 14 类文本的词语tf-idf权重------
-------这里输出第 15 类文本的词语tf-idf权重------
-------这里输出第 16 类文本的词语tf-idf权重------
-------这里输出第 17 类文本的词语tf-idf权重------
-------这里输出第 18 类文本的词语tf-idf权重------
-------这里输出第 19 类文本的词语tf-idf权重------
-------这里输出第 20 类文本的词语tf-idf权重------
-------这里输出第 21 类文本的词语tf-idf权重------
-------这里输出第 22 类文本的词语tf-idf权重------
-------这里输出第 23 类文本的词语tf-idf权重------
-------这里输出第 24 类文本的词语tf-idf权重------
-------这里输出第 25 类文本的词语tf-idf权重------
-------这里输出第 26 类文本的词语tf-idf权重------
-------这里输出

-------这里输出第 231 类文本的词语tf-idf权重------
-------这里输出第 232 类文本的词语tf-idf权重------
-------这里输出第 233 类文本的词语tf-idf权重------
-------这里输出第 234 类文本的词语tf-idf权重------
-------这里输出第 235 类文本的词语tf-idf权重------
-------这里输出第 236 类文本的词语tf-idf权重------
-------这里输出第 237 类文本的词语tf-idf权重------
-------这里输出第 238 类文本的词语tf-idf权重------
-------这里输出第 239 类文本的词语tf-idf权重------
-------这里输出第 240 类文本的词语tf-idf权重------
-------这里输出第 241 类文本的词语tf-idf权重------
-------这里输出第 242 类文本的词语tf-idf权重------
-------这里输出第 243 类文本的词语tf-idf权重------
-------这里输出第 244 类文本的词语tf-idf权重------
-------这里输出第 245 类文本的词语tf-idf权重------
-------这里输出第 246 类文本的词语tf-idf权重------
-------这里输出第 247 类文本的词语tf-idf权重------
-------这里输出第 248 类文本的词语tf-idf权重------
-------这里输出第 249 类文本的词语tf-idf权重------
-------这里输出第 250 类文本的词语tf-idf权重------
-------这里输出第 251 类文本的词语tf-idf权重------
-------这里输出第 252 类文本的词语tf-idf权重------
-------这里输出第 253 类文本的词语tf-idf权重------
-------这里输出第 254 类文本的词语tf-idf权重------
-------这里输出第 255 类文本的词语tf-idf权重------
-------这里输出第 256 类文本的词语tf-idf权重------
-------这里输出第

-------这里输出第 452 类文本的词语tf-idf权重------
-------这里输出第 453 类文本的词语tf-idf权重------
-------这里输出第 454 类文本的词语tf-idf权重------
-------这里输出第 455 类文本的词语tf-idf权重------
-------这里输出第 456 类文本的词语tf-idf权重------
-------这里输出第 457 类文本的词语tf-idf权重------
-------这里输出第 458 类文本的词语tf-idf权重------
-------这里输出第 459 类文本的词语tf-idf权重------
-------这里输出第 460 类文本的词语tf-idf权重------
-------这里输出第 461 类文本的词语tf-idf权重------
-------这里输出第 462 类文本的词语tf-idf权重------
-------这里输出第 463 类文本的词语tf-idf权重------
-------这里输出第 464 类文本的词语tf-idf权重------
-------这里输出第 465 类文本的词语tf-idf权重------
-------这里输出第 466 类文本的词语tf-idf权重------
-------这里输出第 467 类文本的词语tf-idf权重------
-------这里输出第 468 类文本的词语tf-idf权重------
-------这里输出第 469 类文本的词语tf-idf权重------
-------这里输出第 470 类文本的词语tf-idf权重------
-------这里输出第 471 类文本的词语tf-idf权重------
-------这里输出第 472 类文本的词语tf-idf权重------
-------这里输出第 473 类文本的词语tf-idf权重------
-------这里输出第 474 类文本的词语tf-idf权重------
-------这里输出第 475 类文本的词语tf-idf权重------
-------这里输出第 476 类文本的词语tf-idf权重------
-------这里输出第 477 类文本的词语tf-idf权重------
-------这里输出第

-------这里输出第 679 类文本的词语tf-idf权重------
-------这里输出第 680 类文本的词语tf-idf权重------
-------这里输出第 681 类文本的词语tf-idf权重------
-------这里输出第 682 类文本的词语tf-idf权重------
-------这里输出第 683 类文本的词语tf-idf权重------
-------这里输出第 684 类文本的词语tf-idf权重------
-------这里输出第 685 类文本的词语tf-idf权重------
-------这里输出第 686 类文本的词语tf-idf权重------
-------这里输出第 687 类文本的词语tf-idf权重------
-------这里输出第 688 类文本的词语tf-idf权重------
-------这里输出第 689 类文本的词语tf-idf权重------
-------这里输出第 690 类文本的词语tf-idf权重------
-------这里输出第 691 类文本的词语tf-idf权重------
-------这里输出第 692 类文本的词语tf-idf权重------
-------这里输出第 693 类文本的词语tf-idf权重------
-------这里输出第 694 类文本的词语tf-idf权重------
-------这里输出第 695 类文本的词语tf-idf权重------
-------这里输出第 696 类文本的词语tf-idf权重------
-------这里输出第 697 类文本的词语tf-idf权重------
-------这里输出第 698 类文本的词语tf-idf权重------
-------这里输出第 699 类文本的词语tf-idf权重------
-------这里输出第 700 类文本的词语tf-idf权重------
-------这里输出第 701 类文本的词语tf-idf权重------
-------这里输出第 702 类文本的词语tf-idf权重------
-------这里输出第 703 类文本的词语tf-idf权重------
-------这里输出第 704 类文本的词语tf-idf权重------
-------这里输出第

-------这里输出第 898 类文本的词语tf-idf权重------
-------这里输出第 899 类文本的词语tf-idf权重------
-------这里输出第 900 类文本的词语tf-idf权重------
-------这里输出第 901 类文本的词语tf-idf权重------
-------这里输出第 902 类文本的词语tf-idf权重------
-------这里输出第 903 类文本的词语tf-idf权重------
-------这里输出第 904 类文本的词语tf-idf权重------
-------这里输出第 905 类文本的词语tf-idf权重------
-------这里输出第 906 类文本的词语tf-idf权重------
-------这里输出第 907 类文本的词语tf-idf权重------
-------这里输出第 908 类文本的词语tf-idf权重------
-------这里输出第 909 类文本的词语tf-idf权重------
-------这里输出第 910 类文本的词语tf-idf权重------
-------这里输出第 911 类文本的词语tf-idf权重------
-------这里输出第 912 类文本的词语tf-idf权重------
-------这里输出第 913 类文本的词语tf-idf权重------
-------这里输出第 914 类文本的词语tf-idf权重------
-------这里输出第 915 类文本的词语tf-idf权重------
-------这里输出第 916 类文本的词语tf-idf权重------
-------这里输出第 917 类文本的词语tf-idf权重------
-------这里输出第 918 类文本的词语tf-idf权重------
-------这里输出第 919 类文本的词语tf-idf权重------
-------这里输出第 920 类文本的词语tf-idf权重------
-------这里输出第 921 类文本的词语tf-idf权重------
-------这里输出第 922 类文本的词语tf-idf权重------
-------这里输出第 923 类文本的词语tf-idf权重------
-------这里输出第

...

-------这里输出第 9674 类文本的词语tf-idf权重------
-------这里输出第 9675 类文本的词语tf-idf权重------
-------这里输出第 9676 类文本的词语tf-idf权重------
-------这里输出第 9677 类文本的词语tf-idf权重------
-------这里输出第 9678 类文本的词语tf-idf权重------
-------这里输出第 9679 类文本的词语tf-idf权重------
-------这里输出第 9680 类文本的词语tf-idf权重------
-------这里输出第 9681 类文本的词语tf-idf权重------
-------这里输出第 9682 类文本的词语tf-idf权重------
-------这里输出第 9683 类文本的词语tf-idf权重------
-------这里输出第 9684 类文本的词语tf-idf权重------
-------这里输出第 9685 类文本的词语tf-idf权重------
-------这里输出第 9686 类文本的词语tf-idf权重------
-------这里输出第 9687 类文本的词语tf-idf权重------
-------这里输出第 9688 类文本的词语tf-idf权重------
-------这里输出第 9689 类文本的词语tf-idf权重------
-------这里输出第 9690 类文本的词语tf-idf权重------
-------这里输出第 9691 类文本的词语tf-idf权重------
-------这里输出第 9692 类文本的词语tf-idf权重------
-------这里输出第 9693 类文本的词语tf-idf权重------
-------这里输出第 9694 类文本的词语tf-idf权重------
-------这里输出第 9695 类文本的词语tf-idf权重------
-------这里输出第 9696 类文本的词语tf-idf权重------
-------这里输出第 9697 类文本的词语tf-idf权重------
-------这里输出第 9698 类文本的词语tf-idf权重------
-------这里输出第 9699 类文本的词语t

-------这里输出第 9894 类文本的词语tf-idf权重------
-------这里输出第 9895 类文本的词语tf-idf权重------
-------这里输出第 9896 类文本的词语tf-idf权重------
-------这里输出第 9897 类文本的词语tf-idf权重------
-------这里输出第 9898 类文本的词语tf-idf权重------
-------这里输出第 9899 类文本的词语tf-idf权重------
-------这里输出第 9900 类文本的词语tf-idf权重------
-------这里输出第 9901 类文本的词语tf-idf权重------
-------这里输出第 9902 类文本的词语tf-idf权重------
-------这里输出第 9903 类文本的词语tf-idf权重------
-------这里输出第 9904 类文本的词语tf-idf权重------
-------这里输出第 9905 类文本的词语tf-idf权重------
-------这里输出第 9906 类文本的词语tf-idf权重------
-------这里输出第 9907 类文本的词语tf-idf权重------
-------这里输出第 9908 类文本的词语tf-idf权重------
-------这里输出第 9909 类文本的词语tf-idf权重------
-------这里输出第 9910 类文本的词语tf-idf权重------
-------这里输出第 9911 类文本的词语tf-idf权重------
-------这里输出第 9912 类文本的词语tf-idf权重------
-------这里输出第 9913 类文本的词语tf-idf权重------
-------这里输出第 9914 类文本的词语tf-idf权重------
-------这里输出第 9915 类文本的词语tf-idf权重------
-------这里输出第 9916 类文本的词语tf-idf权重------
-------这里输出第 9917 类文本的词语tf-idf权重------
-------这里输出第 9918 类文本的词语tf-idf权重------
-------这里输出第 9919 类文本的词语t

-------这里输出第 10113 类文本的词语tf-idf权重------
-------这里输出第 10114 类文本的词语tf-idf权重------
-------这里输出第 10115 类文本的词语tf-idf权重------
-------这里输出第 10116 类文本的词语tf-idf权重------
-------这里输出第 10117 类文本的词语tf-idf权重------
-------这里输出第 10118 类文本的词语tf-idf权重------
-------这里输出第 10119 类文本的词语tf-idf权重------
-------这里输出第 10120 类文本的词语tf-idf权重------
-------这里输出第 10121 类文本的词语tf-idf权重------
-------这里输出第 10122 类文本的词语tf-idf权重------
-------这里输出第 10123 类文本的词语tf-idf权重------
-------这里输出第 10124 类文本的词语tf-idf权重------
-------这里输出第 10125 类文本的词语tf-idf权重------
-------这里输出第 10126 类文本的词语tf-idf权重------
-------这里输出第 10127 类文本的词语tf-idf权重------
-------这里输出第 10128 类文本的词语tf-idf权重------
-------这里输出第 10129 类文本的词语tf-idf权重------
-------这里输出第 10130 类文本的词语tf-idf权重------
-------这里输出第 10131 类文本的词语tf-idf权重------
-------这里输出第 10132 类文本的词语tf-idf权重------
-------这里输出第 10133 类文本的词语tf-idf权重------
-------这里输出第 10134 类文本的词语tf-idf权重------
-------这里输出第 10135 类文本的词语tf-idf权重------
-------这里输出第 10136 类文本的词语tf-idf权重------
-------这里输出第 10137 类文本的词语tf-idf权重------


-------这里输出第 10330 类文本的词语tf-idf权重------
-------这里输出第 10331 类文本的词语tf-idf权重------
-------这里输出第 10332 类文本的词语tf-idf权重------
-------这里输出第 10333 类文本的词语tf-idf权重------
-------这里输出第 10334 类文本的词语tf-idf权重------
-------这里输出第 10335 类文本的词语tf-idf权重------
-------这里输出第 10336 类文本的词语tf-idf权重------
-------这里输出第 10337 类文本的词语tf-idf权重------
-------这里输出第 10338 类文本的词语tf-idf权重------
-------这里输出第 10339 类文本的词语tf-idf权重------
-------这里输出第 10340 类文本的词语tf-idf权重------
-------这里输出第 10341 类文本的词语tf-idf权重------
-------这里输出第 10342 类文本的词语tf-idf权重------
-------这里输出第 10343 类文本的词语tf-idf权重------
-------这里输出第 10344 类文本的词语tf-idf权重------
-------这里输出第 10345 类文本的词语tf-idf权重------
-------这里输出第 10346 类文本的词语tf-idf权重------
-------这里输出第 10347 类文本的词语tf-idf权重------
-------这里输出第 10348 类文本的词语tf-idf权重------
-------这里输出第 10349 类文本的词语tf-idf权重------
-------这里输出第 10350 类文本的词语tf-idf权重------
-------这里输出第 10351 类文本的词语tf-idf权重------
-------这里输出第 10352 类文本的词语tf-idf权重------
-------这里输出第 10353 类文本的词语tf-idf权重------
-------这里输出第 10354 类文本的词语tf-idf权重------


-------这里输出第 10535 类文本的词语tf-idf权重------
-------这里输出第 10536 类文本的词语tf-idf权重------
-------这里输出第 10537 类文本的词语tf-idf权重------
-------这里输出第 10538 类文本的词语tf-idf权重------
-------这里输出第 10539 类文本的词语tf-idf权重------
-------这里输出第 10540 类文本的词语tf-idf权重------
-------这里输出第 10541 类文本的词语tf-idf权重------
-------这里输出第 10542 类文本的词语tf-idf权重------
-------这里输出第 10543 类文本的词语tf-idf权重------
-------这里输出第 10544 类文本的词语tf-idf权重------
-------这里输出第 10545 类文本的词语tf-idf权重------
-------这里输出第 10546 类文本的词语tf-idf权重------
-------这里输出第 10547 类文本的词语tf-idf权重------
-------这里输出第 10548 类文本的词语tf-idf权重------
-------这里输出第 10549 类文本的词语tf-idf权重------
-------这里输出第 10550 类文本的词语tf-idf权重------
-------这里输出第 10551 类文本的词语tf-idf权重------
-------这里输出第 10552 类文本的词语tf-idf权重------
-------这里输出第 10553 类文本的词语tf-idf权重------
-------这里输出第 10554 类文本的词语tf-idf权重------
-------这里输出第 10555 类文本的词语tf-idf权重------
-------这里输出第 10556 类文本的词语tf-idf权重------
-------这里输出第 10557 类文本的词语tf-idf权重------
-------这里输出第 10558 类文本的词语tf-idf权重------
-------这里输出第 10559 类文本的词语tf-idf权重------


In [40]:
print(df.shape, test_df.shape, corpus.shape)

(8290, 12) (2364, 2) (10654,)


In [41]:
term_weights.sort(key=lambda v:v[1], reverse=True)

In [42]:
len(term_weights)

101234

In [43]:
import re
# alphanum = re.compile('[a-zA-Z0-9]')
alphanum = re.compile('[a-zA-Z]')

In [44]:
# important_words = set(v[0] for v, _ in zip(term_weights, range(3000)) if not alphanum.search(v[0]))
important_words = set(v[0] for v in term_weights if not alphanum.search(v[0]))

In [45]:
# 0 in important_words

In [46]:
len(important_words)

16324

In [47]:
with open('../data/tfidf1.txt', 'w', encoding='utf-8') as f:
    first = True
    for w in important_words:
        if first:
            first = False
        else:
            f.write('\n')
        f.write(w)
    

In [48]:
with open('../data/tfidf1.txt', encoding='utf-8') as f:
    important_words = [line.strip() for line in f.readlines()]

In [49]:
tdf = pd.DataFrame()
test_tdf = pd.DataFrame()
#tdf['content'] = df['content']
# tdf['content_id'] = df['content_id']
for w in important_words:
    tdf[w] = df['content'].apply(lambda text: 1 if text.find(w) != -1 else 0)
    test_tdf[w] = test_df['content'].apply(lambda text: 1 if text.find(w) != -1 else 0)

In [50]:
tdf.head(1)

,寄过来,黄色,空闲,四趟,车多,雪白,再就是,干什么,油到,叽响,...,车坏,该刹,后防,塞得,觉悟,气枪,购得,车真,中度,车龄长
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
invalid_cols = []
for col in tdf.columns:
    if tdf[tdf[col]>0].shape[0] < 5:
        invalid_cols.append(col)
print(len(invalid_cols))

12893


In [55]:
tdf.drop(columns=invalid_cols, inplace=True)
tdf.shape

(8290, 3431)

In [56]:
test_tdf.drop(columns=invalid_cols, inplace=True)
test_tdf.shape

(2364, 3431)

In [57]:
tdf.to_csv('../data/tfidf_train.csv', index=False)
test_tdf.to_csv('../data/tfidf_test.csv', index=False)

In [ ]:


# for index in range(tdf.shape[0]):
#     for j in range(len(word)):
#         if weight[index][j] > 0 and (word[j] in important_words):
#             #print(word[j])
#             tdf.loc[index, word[j]] = weight[index][j]

            

In [ ]:
# tdf.head(5)

In [ ]:
# tdf['空档'].value_counts()[0]

In [ ]:
# meaningless_cols = []
# for col in tdf.columns:
#     if tdf[col].value_counts()[0] > tdf.shape[0]-5:
#         meaningless_cols.append(col)
# print(meaningless_cols)

In [ ]:
# print(len(meaningless_cols))

In [ ]:
# meaningful_cols = list(important_words-set(meaningless_cols))

In [ ]:
# train_tdf = tdf[meaningful_cols]

In [ ]:
# train_tdf.head(3)

In [ ]:
# train_tdf.to_csv('../data/tfidf_train.csv', index=False)
# meaningful_words = set(meaningful_cols)

In [ ]:
# test_tdf = pd.DataFrame()
# for w in meaningful_cols:
#     test_tdf[w] = np.array([0] * test_df.shape[0])

In [ ]:
# for i in range(test_tdf.shape[0]):
#     index = i + train_tdf.shape[0]
#     for j in range(len(word)):
#         if weight[index][j] > 0 and (word[j] in meaningful_words):
#             #print(word[j])
#             test_tdf.loc[i, word[j]] = weight[index][j]

       

In [ ]:
# test_tdf.head(4)

In [ ]:
# test_tdf.to_csv('../data/tfidf_test.csv', index=False)

### 利用规则提取的特征

In [31]:
import rule_utils as ru

In [32]:
subjects = {
    'price': '价格', 
    'interior': '内饰', 
    'power': '动力', 
    'surface': '外观', 
    'safety': '安全性', 
    'operation':'操控', 
    'gas' : '油耗', 
    'space': '空间', 
    'comfort': '舒适性', 
    'config': '配置'
}


In [33]:
matchers = [(sub, ru.match_subject('../rules/{}.txt'.format(sub))) for sub in subjects]

In [34]:
df = pd.read_csv('../data/input.csv')
df.head(5)

,content_id,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
0,01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0
1,01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,02VHu4amZkpAGTb7,常用配件价格不贵，跟大众差不多。而且常用配件都有备货，如果不太常用的可能需要等，但也绝不是从...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,038JPMTIu1tRcZ2p,4万公里，悲催的11。。。市区每天办事，堵成汪。 哈哈哈。如果以前的锐志，起码13~14,3.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0
4,03AUEJcWODxb9ZHa,建议最好使用原厂，subaru的发动机的冷却方式更别的车不一样，所以原厂更适合。,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [40]:
df['content'].shape

(8290,)

In [35]:
test_df = pd.read_csv('../data/test_public.csv')
test_df.head(5)

,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪


In [41]:
rdf = pd.DataFrame()
for sub in subjects:
    rdf[sub] = np.array([0] * df.shape[0])

In [42]:
for i, value in enumerate(df['content']):
    has = False
    for sub, matcher in matchers:          
        if matcher(value):
            rdf.loc[i, sub] = 1              


In [43]:
rdf.shape

(8290, 10)

In [44]:
# rdf.fillna(0, inplace=True)
rdf.head(5)

,price,interior,power,surface,safety,operation,gas,space,comfort,config
0,0,0,0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0


In [45]:
rdf.to_csv('../data/rule_train.csv', index=False) 

In [46]:
test_rdf = pd.DataFrame()
for sub in subjects:
    test_rdf[sub] = np.array([0] * test_df.shape[0])

In [47]:
for i, value in enumerate(test_df['content']):
    has = False
    for sub, matcher in matchers:          
        if matcher(value):
            test_rdf.loc[i, sub] = 1              


In [48]:
# test_rdf.fillna(0, inplace=True)
test_rdf.head(3)

,price,interior,power,surface,safety,operation,gas,space,comfort,config
0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0


In [49]:
test_rdf.to_csv('../data/rule_test.csv', index=False) 

### 合并规则和ｔｆｉｄｆ                                                                                                                                                                                     

In [58]:
tdf = pd.read_csv('../data/tfidf_train.csv')
rdf = pd.read_csv('../data/rule_train.csv')
trdf = pd.merge(tdf, rdf, left_index=True, right_index=True)
trdf.head(3)

,车多,再就是,干什么,没得说,降档,少有,主力,满满的,算是,第一个,...,config_9,config_10,config_11,config_12,config_13,config_14,config_15,config_16,config_17,config_18
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
test_tdf = pd.read_csv('../data/tfidf_test.csv')
test_rdf = pd.read_csv('../data/rule_test.csv')
test_trdf = pd.merge(test_tdf, test_rdf, left_index=True, right_index=True)
test_trdf.head(3)

,车多,再就是,干什么,没得说,降档,少有,主力,满满的,算是,第一个,...,config_9,config_10,config_11,config_12,config_13,config_14,config_15,config_16,config_17,config_18
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
trdf.to_csv('../data/merge_train.csv', index=False)

In [61]:
test_trdf.to_csv('../data/merge_test.csv', index=False)

In [62]:
rdf.shape

(8290, 110)

In [63]:
trdf.shape

(8290, 3541)

In [64]:
tdf.shape

(8290, 3431)

### 使用doc2vec作为特征

In [94]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile

In [87]:
df = pd.read_csv('../data/input.csv')
df.head(5)

,content_id,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
0,01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0
1,01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,02VHu4amZkpAGTb7,常用配件价格不贵，跟大众差不多。而且常用配件都有备货，如果不太常用的可能需要等，但也绝不是从...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,038JPMTIu1tRcZ2p,4万公里，悲催的11。。。市区每天办事，堵成汪。 哈哈哈。如果以前的锐志，起码13~14,3.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0
4,03AUEJcWODxb9ZHa,建议最好使用原厂，subaru的发动机的冷却方式更别的车不一样，所以原厂更适合。,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [91]:
sign_ptn = re.compile(r'[\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。×》〕:《〈？、~@#￥%……&*（）]+')
def pre_doc(text):
    text = sign_ptn.sub(' ', text)
    return jieba.lcut(text)

In [92]:
documents = [TaggedDocument(pre_doc(doc), [i]) for i, doc in enumerate(df['content'])]

In [93]:
model = Doc2Vec(documents, vector_size=300, window=5, min_count=5, workers=4)

In [97]:

fname = get_tmpfile("my_doc2vec_model")
model.save(fname)

In [98]:
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

In [106]:
model.train(documents, total_examples=model.corpus_count, epochs=3)

In [107]:
test_df = pd.read_csv('../data/test_public.csv')

In [116]:
ddf = pd.DataFrame(columns=['dim{}'.format(i) for i in range(300)])

In [117]:
for i, line in enumerate(df['content']):
    vector = model.infer_vector(pre_doc(line))
    ddf.loc[i] = vector

In [118]:
ddf.head(3)

,dim0,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,...,dim290,dim291,dim292,dim293,dim294,dim295,dim296,dim297,dim298,dim299
0,0.034384,-0.016823,0.006522,0.020872,0.006667,0.030710,0.014190,-0.010082,-0.004890,0.033017,...,-0.004739,0.035904,-0.089773,-0.00951,-0.070302,-0.000280,0.005745,-0.006616,0.005799,0.005369
1,0.025111,0.002970,-0.025248,0.036160,-0.023249,-0.041618,-0.021223,-0.010170,-0.017132,0.029365,...,0.016029,-0.013994,0.006455,-0.00063,0.016549,-0.008239,-0.012055,-0.022765,-0.013865,-0.011685
2,0.003015,0.025754,-0.025092,0.037839,-0.035557,-0.035499,-0.007389,0.011180,-0.021014,0.001972,...,-0.030879,-0.002687,-0.021454,-0.01964,-0.009447,0.012752,-0.041384,-0.031563,-0.009785,-0.019306


In [119]:
ddf.to_csv('../data/doc2vec_train.csv', index=False)

In [121]:
test_ddf = pd.DataFrame(columns=['dim{}'.format(i) for i in range(300)])
for i, line in enumerate(test_df['content']):
    vector = model.infer_vector(pre_doc(line))
    test_ddf.loc[i] = vector
test_ddf.head(3)

,dim0,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,...,dim290,dim291,dim292,dim293,dim294,dim295,dim296,dim297,dim298,dim299
0,0.017418,0.005431,0.005071,0.007334,-0.020590,-0.001904,0.008904,-0.018177,-0.026034,-0.008714,...,-0.006851,-0.006440,-0.012720,-0.017694,0.002505,-0.003512,0.008976,-0.015040,0.012093,-0.030001
1,-0.021517,0.029978,-0.026525,0.019812,-0.016152,-0.023428,-0.015707,0.013926,-0.000948,-0.000405,...,0.004583,-0.028824,-0.000966,-0.005942,0.018003,0.008348,0.007940,-0.006643,0.008800,-0.000609
2,-0.026036,0.014354,0.005407,-0.009929,0.004893,-0.002184,0.004948,0.007860,0.012693,0.019432,...,0.016673,-0.001353,-0.017586,0.001380,-0.000243,0.015915,0.025330,0.012687,0.013213,0.007844


In [122]:
test_ddf.to_csv('../data/doc2vec_test.csv', index=False)